In [41]:
import cell2cell as c2c

import numpy as np
import pandas as pd

from collections import defaultdict
import random
import itertools
from tqdm.auto import tqdm

# Setups

In [7]:
rnaseq_setup = dict()
rnaseq_setup['gene_col'] = 'gene_id'
rnaseq_setup['drop_nangenes'] = True
rnaseq_setup['log_transform'] = False

In [8]:
meta_setup = dict()
meta_setup['sample_col'] = '#SampleID'
meta_setup['group_col'] = 'Groups'

In [9]:
ppi_setup = dict()
ppi_setup['protein_cols'] = ['Ligand_WB', 'Receptor_WB']

In [10]:
cutoff_setup = dict()
cutoff_setup['type'] = 'constant_value'
cutoff_setup['parameter'] = 10 # TPM (Changed later)

In [11]:
analysis_setup = dict()
analysis_setup['communication_score'] = 'expression_thresholding'
analysis_setup['cci_score'] = 'count' # Changed later
analysis_setup['cci_type'] = 'undirected'

# Load Data

In [22]:
data_folder = '../../data/'

In [ ]:
import os

output_folder = '../../results/Benchmarking/'
if not os.path.isdir(output_folder):
    os.makedirs(output_folder, exist_ok=True)

In [50]:
files = dict()
files['rnaseq'] = data_folder + '/RNA-Seq/Celegans_RNASeqData_Cell.xlsx'
files['metadata'] = data_folder + '/RNA-Seq/Celegans_cell_metadata.tsv'
files['ppi'] = data_folder + '/PPI-Networks/Celegans-Curated-LR-pairs.xlsx'
files['output_folder'] = output_folder

In [52]:
import os
if not os.path.isdir(files['output_folder']):
    os.mkdir(files['output_folder'])

In [24]:
rnaseq_data = c2c.io.load_rnaseq(rnaseq_file=files['rnaseq'],
                                 gene_column=rnaseq_setup['gene_col'],
                                 drop_nangenes=rnaseq_setup['drop_nangenes'],
                                 log_transformation=rnaseq_setup['log_transform'],
                                 format='auto')

Opening RNAseq datasets from /Users/earmingol/Dropbox/Universidad/UCSanDiego/Lab_Lewis/CCI-Project/Celegans/Project-cell2cell/Celegans-cell2cell-2022/Data//RNA-Seq/Celegans_RNASeqData_Cell.xlsx
/Users/earmingol/Dropbox/Universidad/UCSanDiego/Lab_Lewis/CCI-Project/Celegans/Project-cell2cell/Celegans-cell2cell-2022/Data//RNA-Seq/Celegans_RNASeqData_Cell.xlsx was correctly loaded


In [25]:
meta = c2c.io.load_metadata(metadata_file=files['metadata'],
                            cell_labels=rnaseq_data.columns,
                            format='auto')

/Users/earmingol/Dropbox/Universidad/UCSanDiego/Lab_Lewis/CCI-Project/Celegans/Project-cell2cell/Celegans-cell2cell-2022/Data//RNA-Seq/Celegans_cell_metadata.tsv was correctly loaded


In [26]:
ppi_data = c2c.io.load_ppi(ppi_file=files['ppi'],
                           interaction_columns=ppi_setup['protein_cols'],
                           rnaseq_genes=list(rnaseq_data.index),
                           format='auto')

Opening PPI datasets from /Users/earmingol/Dropbox/Universidad/UCSanDiego/Lab_Lewis/CCI-Project/Celegans/Project-cell2cell/Celegans-cell2cell-2022/Data//PPI-Networks/Celegans-Curated-LR-pairs.xlsx
/Users/earmingol/Dropbox/Universidad/UCSanDiego/Lab_Lewis/CCI-Project/Celegans/Project-cell2cell/Celegans-cell2cell-2022/Data//PPI-Networks/Celegans-Curated-LR-pairs.xlsx was correctly loaded
Removing bidirectionality of PPI network
Simplying PPI network


In [27]:
lr_pairs = c2c.io.load_table(files['ppi'])

/Users/earmingol/Dropbox/Universidad/UCSanDiego/Lab_Lewis/CCI-Project/Celegans/Project-cell2cell/Celegans-cell2cell-2022/Data//PPI-Networks/Celegans-Curated-LR-pairs.xlsx was correctly loaded


In [28]:
lr_genes = list(set(lr_pairs['Ligand_WB'].unique()).union(set(lr_pairs['Receptor_WB'].unique())))

In [29]:
excluded_cells = ['Distal_tip_cells',
                  'Sex_myoblasts',
                  'Socket_cells',
                  'Vulval_precursors',
                  'flp-1(+)_interneurons']

In [30]:
included_cells = list((set(rnaseq_data.columns)) - set(excluded_cells))

In [31]:
rnaseq_data = rnaseq_data[included_cells]

# Analysis

In [32]:
if analysis_setup['cci_type'] == 'undirected':
    bi_ppi_data = c2c.preprocessing.bidirectional_ppi_for_cci(ppi_data=ppi_data, verbose=False)
    ref_ppi = ppi_data
else:
    bi_ppi_data = ppi_data.copy()
    ref_ppi = None

In [49]:
tpm_values = [5, 10, 50, 100]

In [47]:
cci_scores = ['bray_curtis', 'count']

In [61]:
# Smillie setups
permutations = 9999
seed = 888

In [63]:
for score in cci_scores:
    analysis_setup['cci_score'] = score
    for tpm in tpm_values:
        cutoff_setup['parameter'] = tpm
        
        # Base scores
        interaction_space = c2c.core.InteractionSpace(rnaseq_data=rnaseq_data[included_cells],
                                                      ppi_data=bi_ppi_data,
                                                      gene_cutoffs=cutoff_setup,
                                                      communication_score=analysis_setup['communication_score'],
                                                      cci_score=analysis_setup['cci_score'],
                                                      cci_type=analysis_setup['cci_type'],
                                                      verbose=False)

        # Compute interactions
        interaction_space.compute_pairwise_cci_scores(use_ppi_score=False, verbose=False)
        cci_matrix = interaction_space.interaction_elements['cci_matrix'].copy()
        
        cci_matrix.to_csv(files['output_folder'] + 'CCI-Matrix-{}-{}-TPM.csv'.format(score.capitalize(), tpm))
        
        
        ## Compute Smillie when score == 'count' ##
        if score == 'count':
            # Generate degree distribution for permutation
            rnaseq_df = rnaseq_data.loc[lr_genes, :]
            
            cell_degrees = rnaseq_df.gt(cutoff_setup['parameter']).sum().sort_values()

            min_val = rnaseq_df.gt(cutoff_setup['parameter']).sum().min()
            max_val = rnaseq_df.gt(cutoff_setup['parameter']).sum().max()

            degree_groups = defaultdict(set)
            step = 10 # Bin size for same distribution

            bins = np.arange(min_val, max_val, step)
            assignment = np.digitize(cell_degrees.values, bins, right=False)

            for i, k in enumerate(assignment):
                degree_groups[k].add(list(cell_degrees.index)[i])

            degree_groups = {k : list(v) for k, v in degree_groups.items()}

            # Permutations
            og_list = list(itertools.chain(*degree_groups.values()))
            randomized_scores = []

            for i in tqdm(range(permutations)):

                shuffled_groups = []

                for v in degree_groups.values():
                    random.seed(seed+i)
                    shuffled_groups.extend(random.sample(v, len(v)))

                cell_mapper = dict(zip(og_list, shuffled_groups))

                shuffled_rnaseq = rnaseq_df.copy()
                shuffled_rnaseq = shuffled_rnaseq.rename(columns=cell_mapper)
                shuffled_rnaseq = shuffled_rnaseq[included_cells] # Order of labels is important

                # Compute CCI scores
                interaction_space = c2c.core.InteractionSpace(rnaseq_data=shuffled_rnaseq,
                                                          ppi_data=bi_ppi_data,
                                                          gene_cutoffs=cutoff_setup,
                                                          communication_score=analysis_setup['communication_score'],
                                                          cci_score=analysis_setup['cci_score'],
                                                          cci_type=analysis_setup['cci_type'],
                                                          verbose=False)

                # Compute interactions
                interaction_space.compute_pairwise_cci_scores(use_ppi_score=False, verbose=False)

                randomized_scores.append(interaction_space.interaction_elements['cci_matrix'].values.flatten())

            # Add base scores as part of the null distribution
            base_scores = cci_matrix.values.flatten()
            randomized_scores.append(base_scores)
            randomized_scores = np.array(randomized_scores)

            # Compute P-values
            pvals = np.ones(base_scores.shape)

            for i in range(len(base_scores)):
                dist = randomized_scores[:, i]
                pvals[i] = c2c.stats.permutation.compute_pvalue_from_dist(obs_value=base_scores[i],
                                                                dist=dist,
                                                                consider_size=True,
                                                                comparison='upper'
                                                                )
            pval_df = pd.DataFrame(pvals.reshape(cci_matrix.shape), index=cci_matrix.index, 
                                   columns=cci_matrix.columns)

            output_df = -np.log10(pval_df)
            
            output_df.to_csv(files['output_folder'] + 'CCI-Matrix-Smillie-{}-TPM.csv'.format(tpm))
            output_df.to_csv(data_folder + '/Benchmarking/CCI-Matrix-Smillie-{}-TPM.csv'.format(tpm))

  0%|          | 0/9999 [00:00<?, ?it/s]

  0%|          | 0/9999 [00:00<?, ?it/s]

  0%|          | 0/9999 [00:00<?, ?it/s]

  0%|          | 0/9999 [00:00<?, ?it/s]